In [1]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
import os

# path to your shapefile 
shapefile_path = 'C:/Users/say70/Desktop/NX/2020/6_15/shapefiles/polygons.shp'

# paths to band images
band_paths = [
    'C:/Users/say70/Desktop/NX/2020/6_15/raw/6_15_2020_B.tif',
    'C:/Users/say70/Desktop/NX/2020/6_15/raw/6_15_2020_G.tif',
    'C:/Users/say70/Desktop/NX/2020/6_15/raw/6_15_2020_R.tif',
    'C:/Users/say70/Desktop/NX/2020/6_15/raw/6_15_2020_RE.tif',
    'C:/Users/say70/Desktop/NX/2020/6_15/raw/6_15_2020_NIR.tif'
]

# Band indices
band_names = ['B', 'G', 'R', 'RE', 'NIR']

# Read Shapefile using geopandas
gdf = gpd.read_file(shapefile_path)

# initialize Arrays to store each clipped band
clipped_bands = []

# Initialize list to store individual band paths
individual_band_paths = []

# Create output directory
output_dir = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/Shape clip/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop through each band image and clip it individually
for i, band_path in enumerate(band_paths):
    with rasterio.open(band_path) as src:
        # Clip band to the shapefile geometry
        clipped_band, clipped_transform = mask(src, gdf.geometry, crop=True)
        clipped_bands.append(clipped_band)

        # Specify output path for each individual band using the band name
        band_name = band_names[i]
        individual_band_path = os.path.join(output_dir, f'6_15_2020_{band_name}_clipped.tif')
        individual_band_paths.append(individual_band_path)

        # Create a new raster file with the same dimensions as the clipped band
        with rasterio.open(individual_band_path, 'w', driver='GTiff', width=clipped_band.shape[2], height=clipped_band.shape[1], count=1, dtype=clipped_band.dtype, crs=src.crs, transform=clipped_transform) as dst:
            dst.write(clipped_band)
        print(f'Band {band_name} saved to {individual_band_path}')
        
        # Print shape and CRS (sanity check)
        print(f'Shape: {clipped_band.shape}, CRS: {src.crs}')

# stack clipped bands into a single 5-band image
stacked_image = np.squeeze(np.stack(clipped_bands, axis=0))

# Specify output path for the stacked image
stacked_image_output_path = os.path.join(output_dir, '6_15_2020_clipped_stack.tif')

# Create new raster file with the same dimensions as one of the input bands
with rasterio.open(stacked_image_output_path, 'w', driver='GTiff', width=stacked_image.shape[2], height=stacked_image.shape[1], count=stacked_image.shape[0], dtype=stacked_image.dtype, crs=src.crs, transform=clipped_transform) as dst:
    # Write each band separately
    for i in range(stacked_image.shape[0]):
        band_name = band_names[i]
        dst.write(stacked_image[i], i + 1)

print(f'Stacked image saved to {stacked_image_output_path}')

# Print shape and CRS for the stacked image (sanity chekc)
print(f'Stacked image shape: {stacked_image.shape}, CRS: {src.crs}')

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# Sanity check for proper clipping (by merging shape file on the top of the stacked images)

import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio.plot as rplt
from matplotlib.cm import ScalarMappable 

#path to your stacked image
stacked_image_path = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/Shape clip/6_15_2020_clipped_stack.tif'

# Open the stacked image to get the transform and CRS
with rasterio.open(stacked_image_path) as src:
    stacked_image = src.read()
    stacked_transform = src.transform
    stacked_crs = src.crs

# Create a Geodata-frame with the clipped geometry
gdf_clipped = gpd.GeoDataFrame({'geometry': [g for g in gdf.geometry]}, crs=gdf.crs)

# GIVE BAND NAMES
band_names = ['B', 'G', 'R', 'RE', 'NIR']

# Plot each band image with the CRS coordinates and their band names
num_bands = stacked_image.shape[0]
fig, axes = plt.subplots(num_bands, 1, figsize=(15, 10*num_bands))


for i in range(num_bands):
    ax = axes[i]
    band = stacked_image[i]
    im = rplt.show(band, transform=stacked_transform, ax=ax, cmap='viridis')  # You can specify a colormap (e.g., 'viridis')
    gdf_clipped.boundary.plot(ax=ax, color='red')  # Add the shapefile boundaries for reference
    ax.set_title(f"Band {band_names[i]} in CRS Coordinates")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    
    # ScalarMappable for the colormap
    sm = ScalarMappable(cmap='viridis')
    sm.set_array(band)
    
    #colorbar
    cbar = plt.colorbar(sm, ax=ax, orientation='vertical',shrink=0.1)
    cbar.set_label('Pixel Value')

plt.tight_layout()
plt.show()

In [ ]:
#clip all the locations from each band and create a stack of all bands with respective to their location.

import geopandas as gpd
import rasterio
from rasterio.mask import mask
import os
import numpy as np

# path to your shapefile and raster images for all bands
shapefile_path = 'C:/Users/say70/Desktop/NX/2020/6_15/shapefiles/polygons.shp'
bands = ['B', 'G', 'R', 'RE', 'NIR']
band_paths = {
    'B': 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/Shape clip/6_15_2020_B_clipped.tif',
    'G': 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/Shape clip/6_15_2020_G_clipped.tif',
    'R': 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/Shape clip/6_15_2020_R_clipped.tif',
    'RE': 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/Shape clip/6_15_2020_RE_clipped.tif',
    'NIR': 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/Shape clip/6_15_2020_NIR_clipped.tif'
}

# Read the shapefile using geopandas
gdf = gpd.read_file(shapefile_path)
gdf = gdf.sort_values(by='location', ascending=True)

# specify output directory for the stacked images of each location
output_dir = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/clip locations stack/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#create empty list to store stacked images for each location (can be helpful to read all location images using list)
normal_maps=[]
    
# loop through the selected polygons (locations)
for polygon_idx, polygon in enumerate(gdf.geometry):
    # Initialize an empty list to store the clipped images for each band
    clipped_images = []

    # Loop through each band and clip the image for the current location
    for band_name, band_image_path in band_paths.items():
        # Extract the geometry of the current location
        polygon_geometry = gdf.iloc[polygon_idx]['geometry']

        # Open the band image using rasterio
        with rasterio.open(band_image_path) as src:
            # Clip the image to the current location's geometry and specify the output CRS
            clipped_image, clipped_transform = mask(src, [polygon_geometry], crop=True)
            clipped_meta = src.meta.copy()
            clipped_meta['crs'] = {'init': 'epsg:3857'}

        # Append the clipped image to the list
        clipped_images.append(clipped_image[0])  # Extract the first band

    # Stack the clipped images for all bands into a single image
    stacked_image = np.stack(clipped_images, axis=0)

    # Define the output file path for the stacked image
    output_image_path = os.path.join(output_dir, f'location{polygon_idx + 1}_stacked.tif')

    # Create a new raster file with the same dimensions as the clipped images
    with rasterio.open(output_image_path, 'w', driver='GTiff', width=stacked_image.shape[2], height=stacked_image.shape[1], count=stacked_image.shape[0], dtype=stacked_image.dtype, crs=clipped_meta['crs'], transform=clipped_transform) as dst:
        # Write the stacked image
        dst.write(stacked_image)
        
     # Append the stacked image to the list
    normal_maps.append(stacked_image)

print("Stacked images saved in the output directory for each location.")


In [ ]:
# sanity check by plotting the stacked image of particular location

import os
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.windows import Window
from matplotlib.cm import ScalarMappable

#  path to cliped location stacked images
stacked_images_dir = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/clip locations stack/'

#  location index (your choice)
location_index = 1

# Band names corresponding to the order 
band_names = ['B', 'G', 'R', 'RE', 'NIR']

# construct the filename for the selected location
location_filename = f'location{location_index}_stacked.tif'
location_filepath = os.path.join(stacked_images_dir, location_filename)

# check if the selected location file exists
if os.path.exists(location_filepath):
    # open the location stacked image using rasterio
    with rasterio.open(location_filepath) as src:
        # Get the CRS (coordinate reference system) of the source image
        src_crs = src.crs
        
        # Loop through each band and plot it separately
        for band_index in range(src.count):
            band = src.read(band_index + 1, window=Window(0, 0, src.width, src.height))
            
            # Create a new figure and axis for each band
            fig, ax = plt.subplots(figsize=(8, 8))
            
            # Plot the band using rasterio's show function
            show(band, cmap='viridis', transform=src.transform, ax=ax)  # Use the source CRS for plotting
            
            # title and labels for the plot
            ax.set_title(f'Band {band_names[band_index]}')  # Set the band name as the title
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')
            
            #  ScalarMappable for the colorbar
            sm = ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=band.min(), vmax=band.max()))
            sm.set_array([])
            
            # coluorbar to the axis
            cbar = plt.colorbar(sm, ax=ax, label='Pixel Value', shrink=0.15)
            
            # Display 
            plt.show()
else:
    print(f"The location file {location_filepath} does not exist.")
